In [ ]:
! pip install music21

In [ ]:
import glob
import pickle
import time
import gc
from collections import deque
import numpy as np
import pandas as pd
from music21 import converter, note, chord
import tensorflow as tf

from os import listdir

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# Sanity check to make sure the lofi files are accessible.
# You may need to right click the folder on Google Drive and click `Add shortcut
#   to Drive` to make the folder accessible from your own drive, then remount
print(listdir('/content/drive/My Drive/RNN_lofi'))

# Output should be 
#   Mounted at /content/drive/
#   ['lofi', 'lofi_h5']

Mounted at /content/drive/
['lofi_h5', 'lofi', 'allNotes1-old.p', 'allNotes2-old.p', 'allNotes1.p', 'allNotes2.p']


## Processing the midi into an notes array (done)

In [ ]:
def midis_to_notes(midi_files: [str], output_filename: str) -> str:
    # This list will hold notes and chords from all songs appended together
    #   use deque to speedup
    allNotes = deque()

    print('Starting conversion of {} files'.format(len(midi_files)))

    gc.disable() # disable gc to speed up append
    prevtime = time.clock()
    for i, file in enumerate(midi_files):
        #print(i,file)
        midi = converter.parse(file)
        elements = midi.flat.notes

        for elem in elements:
            # Notes look like 'F#3'
            if isinstance(elem, note.Note):
                allNotes.append(elem.pitch.nameWithOctave)
            # Chords look like 'C4.E4.G#4'
            elif isinstance(elem, chord.Chord):
                allNotes.append('.'.join(pitch.nameWithOctave for pitch in elem.pitches))

        if (i+1) % 50 == 0:
            currtime = time.clock()
            print('Finished converting {} files, took {} seconds'.format(i+1, currtime - prevtime))
            prevtime = currtime
            gc.collect()
    gc.enable()

    print('Done converting midis to string array of notes and chords with len: {}'.format(len(allNotes)))

    # Saving the allNotes list to Google Drive cuz it takes bare time to complete
    filepath = '/content/drive/My Drive/RNN_lofi/{}.p'.format(output_filename)
    with open(filepath, 'wb') as pickleFile:
        pickle.dump(allNotes, pickleFile)
    
    return filepath

fileIterator = glob.glob('/content/drive/My Drive/RNN_lofi/lofi/*.midi')
# Looks like the original fileIterator is too big? gets big slow downs around halfway
fileIt1 = fileIterator[:len(fileIterator)//2]
fileIt2 = fileIterator[len(fileIterator)//2:]

filePath1 = midis_to_notes(fileIt1, 'allNotes1')
filePath2 = midis_to_notes(fileIt2, 'allNotes2')


Starting conversion of 285 files


##Processing the notes array into the numpy array format for the NN

In [ ]:
# Loading allNotes from file
allNotes = []

filePath1 = '/content/drive/My Drive/RNN_lofi/allNotes1.p'
filePath2 = '/content/drive/My Drive/RNN_lofi/allNotes2.p'

with open(filePath1, 'rb') as pickleFile1:
    allNotes += pickle.load(pickleFile1)
with open(filePath2, 'rb') as pickleFile2:
    allNotes += pickle.load(pickleFile2)

# Look at 10 previous notes to make a prediction
#   We can tune this parameter if needed, based on the length of 
#   chord progressions
seqLength = 10
print('Using sequence length of {}'.format(seqLength))

pitchSet = sorted(set(allNotes))
numPitches = len(pitchSet)
# here pitches are either notes or chords
#   they are sorted lexicographically, so a chord 'C4.E4' will come after a
#   note 'C4'
print('Identified {} pitches'.format(numPitches))

# Map each note/chord to a number normalized to (0,1)
pitchMapping = dict((note, number) for (number, note) in enumerate(pitchSet))

networkInput = []
networkOutput = []

print('Starting sequencing of {} notes'.format(len(allNotes)))
for i in range(0, len(allNotes)- seqLength):
    sequenceIn = allNotes[i:i+seqLength]
    predictionOut = allNotes[i+seqLength]

    networkInput.append([pitchMapping[note] for note in sequenceIn])
    networkOutput.append(pitchMapping[predictionOut])

    if (i+1) % 100000 == 0:
        print('Finished making {} sequences'.format(i+1))

numSeqs = len(networkInput)
# reshape input to match the LSTM layer format
networkInput = np.reshape(networkInput, (numSeqs, seqLength, 1))
networkInput = networkInput / numPitches

networkOutput = np.array(networkOutput)

print('Done preparing network input and output')
# Now data should be in the desired format for the NN

print(networkInput)

Using sequence length of 10
Identified 42308 pitches
Starting sequencing of 1607745 notes
Finished making 100000 sequences
Finished making 200000 sequences
Finished making 300000 sequences
Finished making 400000 sequences
Finished making 500000 sequences
Finished making 600000 sequences
Finished making 700000 sequences
Finished making 800000 sequences
Finished making 900000 sequences
Finished making 1000000 sequences
Finished making 1100000 sequences
Finished making 1200000 sequences
Finished making 1300000 sequences
Finished making 1400000 sequences
Finished making 1500000 sequences
Finished making 1600000 sequences
Done preparing network input and output
[[[0.3081923 ]
  [0.67188239]
  [0.23120923]
  ...
  [0.19069679]
  [0.34544294]
  [0.92124421]]

 [[0.67188239]
  [0.23120923]
  [0.95298761]
  ...
  [0.34544294]
  [0.92124421]
  [0.3594592 ]]

 [[0.23120923]
  [0.95298761]
  [0.46875295]
  ...
  [0.92124421]
  [0.3594592 ]
  [0.20218398]]

 ...

 [[0.67188239]
  [0.24818001]
  [0.

In [ ]:
print(type(networkOutput))

<class 'numpy.ndarray'>


# Model


In [ ]:
# Baseline from: https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5?fbclid=IwAR34r6YILe8VuG56VukdnhoM8GnwQUE7OEz3FofTtEVZwqsLYkBiSl-JGSE

import numpy as np
import tensorflow as tf
from numpy import array
from tensorflow import keras
from tensorflow.keras import layers

networkInputSmall = networkInput[:10000]
networkOutputSmall = networkOutput[:10000]


# Things we need to figure out. Lot of these are probably hyperparameters:
# 1. number of internal units for layers.
# 2. what is dense and what to do with it
# 3. input/output dimensions
# 4. for each layer and tool we use, look into the possible arguments and figure out what we need to do with each


model = keras.Sequential()
model.add(layers.LSTM(
    512,
    input_shape=(networkInputSmall.shape[1], networkInputSmall.shape[2]),
    return_sequences=True
))
model.add(layers.Dropout(0.3))

# Add LSTM layer with 512 internal units. We can define stuff like activation function here too.
# Comes built in with the information gating system we talk about in our report according to:
# https://towardsdatascience.com/implementation-of-rnn-lstm-and-gru-a4250bf6c090
model.add(layers.LSTM(512, return_sequences=True))
model.add(layers.Dropout(0.3))
model.add(layers.LSTM(512))
model.add(layers.Dense(256))
model.add(layers.Dropout(0.3))
# Add a dense layer with 10 units. A dense layer just means it gets output from all neurons in the previous layer.
model.add(layers.Dense(numPitches))
model.add(layers.Activation('softmax'))

model.add(layers.Dense(1))

# Prints overview of model.
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=1,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]


model.fit(networkInputSmall, networkOutputSmall, epochs=1, batch_size=64, callbacks=callbacks_list)


modelFilepath = '/content/drive/My Drive/RNN_lofi/trained_model'
with open(filepath, 'wb') as pickleFile:
  pickle.dump(model, pickleFile)



Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 10, 512)           1052672   
_________________________________________________________________
dropout_36 (Dropout)         (None, 10, 512)           0         
_________________________________________________________________
lstm_37 (LSTM)               (None, 10, 512)           2099200   
_________________________________________________________________
dropout_37 (Dropout)         (None, 10, 512)           0         
_________________________________________________________________
lstm_38 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_34 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_38 (Dropout)         (None, 256)             

TypeError: ignored

# Generate Music with Model

In [ ]:
from music21 import converter, instrument, note, chord, stream

start = np.random.randint(0, len(networkInputSmall)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchSet))
pattern = networkInputSmall[start]
prediction_output = []

# generate 500 notes
for note_index in range(100):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(numPitches)
    prediction = model.predict(prediction_input, verbose=1)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]


offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    print(pattern)
    # pattern is a chord
    if ('.' in pattern[0][0]):
        notes_in_chord = pattern[0][0].split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern[0][0])
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

1/1 [==============================] - 0s 2ms/step
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0
A0


'test_output.mid'